In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

## Data Sources

SparkSQL supports operating on a veriety of data sources through the DataFrame interface

We'll exporle genral methods for loading and saving data suing the Spark Data Sources and then go into specific optins that are avalable for the build-in data sources. 

### Genric load/save command

In [2]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder.appName("Datasources").getOrCreate()

In [17]:
df = spark.read.load("salary_data.parquet")

In [18]:
df.show()

+----------+----------+---------+------+
|Department|Designtion|FirstName|Salary|
+----------+----------+---------+------+
|     Admin|   Maneger|   Michel|  1000|
|     Sales|    Junior|  Faraday|   900|
| Marketing| Porfessor|   Newton|  1100|
|     Sales| President|    James|  1000|
|     Sales|   Maneger|     Bond|   700|
|     Admin| President|   Donald|   900|
| Marketing|    Junior|    Trump|   600|
|     Admin|    Junior|    Rocco|   950|
| Marketing| President|    Ethan|  1200|
|     Admin|    Intern|     Hunt|   400|
+----------+----------+---------+------+



parquet is a columnar format that is supported by most of the big data processing systems.

In [ ]:
df.select ("Department","Designtion","Salary").write.save("anonymous_data.parquet")

### Manually Specifying Options

In [19]:
df = spark.read.load("employees.json", format = "json")

In [20]:
df.show() 

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Borta|  4000|
|  David|  4800|
|  Sandy|  5100|
|  Peter|  1500|
|   John|  4700|
+-------+------+



In [21]:
df = spark.read.load("profession.csv", format = "csv", inferSchema = True, header = True)

### options

format to specify the file format 

inferScema to make spark figure out the data schema

header tells spark that the file has column title row

In [22]:
df.show()

+------+---+-----------------+
|  name|age|            title|
+------+---+-----------------+
|  john| 30|           doctor|
|  Boby| 32|        Developer|
| Peter| 29|        Scientist|
| Kevin| 32|           Police|
|Andrew| 37|      gym trainer|
| Kumar| 40|software engineer|
|   Tom| 51|             Chef|
+------+---+-----------------+



In [ ]:
df.write.save("random.parquet", format = "parquet")

### SQL queries directly be run on files 

instead of the table we can specify the table path.

In [23]:
df = spark.sql("select * from parquet.`salary_data.parquet`")

In [24]:
df.show()

+----------+----------+---------+------+
|Department|Designtion|FirstName|Salary|
+----------+----------+---------+------+
|     Admin|   Maneger|   Michel|  1000|
|     Sales|    Junior|  Faraday|   900|
| Marketing| Porfessor|   Newton|  1100|
|     Sales| President|    James|  1000|
|     Sales|   Maneger|     Bond|   700|
|     Admin| President|   Donald|   900|
| Marketing|    Junior|    Trump|   600|
|     Admin|    Junior|    Rocco|   950|
| Marketing| President|    Ethan|  1200|
|     Admin|    Intern|     Hunt|   400|
+----------+----------+---------+------+



### Now let's take a look at how the we load and save specific files 

## 1. json 

Spark SQL can automatically infer the schema of a JSON dataset and load it as a DataFrame. 

This conversion can be done using SparkSession.read.json on a JSON file.

In [25]:
df = spark.read.json("employees.json")

In [26]:
df.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Borta|  4000|
|  David|  4800|
|  Sandy|  5100|
|  Peter|  1500|
|   John|  4700|
+-------+------+



## 2. Parquet files 

In [27]:
df = spark.read.parquet("salary_data.parquet")

In [28]:
df.show()

+----------+----------+---------+------+
|Department|Designtion|FirstName|Salary|
+----------+----------+---------+------+
|     Admin|   Maneger|   Michel|  1000|
|     Sales|    Junior|  Faraday|   900|
| Marketing| Porfessor|   Newton|  1100|
|     Sales| President|    James|  1000|
|     Sales|   Maneger|     Bond|   700|
|     Admin| President|   Donald|   900|
| Marketing|    Junior|    Trump|   600|
|     Admin|    Junior|    Rocco|   950|
| Marketing| President|    Ethan|  1200|
|     Admin|    Intern|     Hunt|   400|
+----------+----------+---------+------+



##  JDBC to other databases 

Spark sql can read data from other databases using JDBC. 

The output of this read operation is in the form of a dataframe. Which can easity be processed in Spark SQL or joined with other data sources.  

Users can specify the JDBC connection properties in the data source options. user and password are normally provided as connection properties for logging into the data sources.

In [ ]:
jdbcDF3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .load()

above code is an example fo connecting with a external datbase. 

format  - tells that is database. 

url - specifies the address of the database

dbtable - name of the table is to be used 

user - username to sign in to the database 

password - password for siging in

##### These are normally used options there are a lot of other options like this. We can expore them in the documentation. 

## 4. Hive Tables 

Sqprk SQL also supports running Apache Hive query. 

Hive tabels have a lot of dependencies. Spark cannot infer these dependencies on its own. Having said that is we bring all these dependencies in the classpath of spark. Then Spark can fingure it out. 

For cluster applications these dependencies also need to be present on all the nodes. As these nodes will also need to seralise and deserealise the objects. 

##### When working with HIVE we need to instanciate a spark session with HIVE support 

In [3]:
spark_hive = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
spark_hive

##### enableHiveSupport() 

This is the method which we need to call when bulding the SparkSession 

###### After connecting HIVE

Spark is connected to HIVE. Users can send queries in the same format as an SQL query. 

In [6]:
spark_hive.sql("CREATE TABLE IF NOT EXISTS company (company STRING, sales int) USING hive")

DataFrame[]

In [8]:
spark_hive.sql("LOAD DATA LOCAL INPATH 'sales_info.txt' INTO TABLE company")

DataFrame[]

In [10]:
spark_hive.sql("select * from company").show()

+------------+-----+
|     company|sales|
+------------+-----+
|   Google_Q1|  200|
|   Google_Q2|  120|
|   Google_Q3|  340|
|   Google_Q4|  300|
|Microsoft_Q2|  300|
|Microsoft_Q3|  124|
|Microsoft_Q4|  243|
|Microsoft_Q1|  330|
| Facebook_Q3|  870|
| Facebook_Q2|  750|
|    Apple_Q1|  650|
|    Apple_Q2|  550|
|    Apple_Q3|  750|
|    Apple_Q4|  850|
+------------+-----+



In [12]:
df_hive = spark_hive.sql("select * from company")

### Table partitioning 

This is a common optimisation approch used like Hive. In a partitioned table, data is usually stored in different directoried, with partitioning column values encoded in the path of each partition directiry. 

All built-in file sources (including Text/CSV/JSON/ORC/parquet) are able to discover and infer partitioning information automatically.

example : date